In [1]:
!pip install -Uq llama-index
!pip install -Uq llama-index-embeddings-huggingface

!pip install -Uq auto-gptq
!pip install -Uq optimum
!pip install -Uq bitsandbytes

!pip install -Uq transformers
!pip install -Uq peft
!pip install -Uq accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

import os, torch

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HF_TOKEN")

2024-07-15 05:52:45.152576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 05:52:45.152679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 05:52:45.325761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
Settings.embed_model = HuggingFaceEmbedding(model_name="WhereIsAI/UAE-Large-V1")
Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/65.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [4]:
documents = SimpleDirectoryReader("/kaggle/input/rag-project/RAG_project").load_data()
print(len(documents))
print(type(documents))

86
<class 'list'>


In [5]:
print(documents[65].text)

162 PHYSICS
/trianglert/trianglertvary fr om one point of the body to the other . If
the body is so extended that g varies fr om part
to part of the body, then the centr e of gravity
and centr e of mass will not coincide. Basically,
the two ar e different concepts. The centr e of
mass has nothing to do with gravity. It depends
only on the distribution of mass of the body.
In Sec. 7.2 we found out the position of the
centr e of mass of several r egular , homogeneous
objects. Obviously the method used ther e gives
us also the centr e of gravity of these bodies, if
they ar e small enough.
Figur e 7.25 illustrates another way of
deter mining the CG of an r egular shaped body
like a car dboar d. If you suspend the body fr om
some point like A, the vertical line thr ough A
passes thr ough the CG. W e mark the vertical
AA1. We then suspend the body thr ough other
points like B and C. The intersection of the
verticals gives the CG. Explain why the method
works. Since the body is small enough, 

In [6]:
print(documents[11].text)

176 Chemistry7.77.77.77.77.7 PhosphinePhosphinePhosphinePhosphinePhosphineIt is polymeric, consisting of chains of P4
tetrahedra linked together in the manner as shown
in Fig. 7.3.
Black phosphorus  has two forms α-black
phosphorus and β-black phosphorus. α-Black
phosphorus is formed when red phosphorus is
heated in a sealed tube at 803K. It can be sublimed
in air and has opaque monoclinic or rhombohedral
crystals. It does not oxidise in air . β-Black phosphorus is pr epared by
heating white phosphorus at 473 K under high pressure. It does not
burn in air upto 673 K.
Preparation
Phosphine is prepared by the reaction of calcium phosphide with water
or dilute HCl.
Ca3P2 + 6H2O → 3Ca(OH)2 + 2PH3
Ca3P2 + 6HCl → 3CaCl2 + 2PH3
In the laboratory, it is prepared by heating white phosphorus with
concentrated NaOH solution in an inert atmosphere of CO2.
( )4 2 3 2 2P 3NaOH 3H O PH 3NaH PO
sodium hypophosphite+ + → +
When pure, it is non inflammable but becomes inflammable owing
to the presence o

In [7]:
index = VectorStoreIndex.from_documents(documents)

top_k = 2

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [10]:
query = "What are the anomalous properties of nitrogen?"
response = query_engine.query(query)
print(response.source_nodes[0].text, "\n\n", response.source_nodes[1].text, "\n\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

However
+3 oxidation state in case of arsenic, antimony and bismuth becomes
increasingly stable with respect to disproportionation.
Nitrogen is restricted to a maximum covalency of 4 since only four
(one s and three p) orbitals are available for bonding. The heavier elements
have vacant d orbitals in the outermost shell which can be used for
bonding (covalency) and hence, expand their covalence as in P F–
6.
Anomalous properties of nitrogen
Nitrogen differs from the rest of the members of this group due to
its small size, high electronegativity, high ionisation enthalpy and
non-availability of d orbitals. Nitrogen has unique ability to form
pπ ππ ππ-pπ ππ ππ multiple  bonds with itself and with other elements having
small size and high electronegativity (e.g., C, O). 

 168 Chemistry
nitrogen. Another factor which affects the chemistry of nitrogen is
the absence of d orbitals in its valence shell. Besides restricting its
covalency to four , nitrogen cannot for m dπ ππ ππ–pπ ππ ππ bond 

In [11]:
!huggingface-cli login --token $secret_hf

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [12]:
model_name = "meta-llama/Meta-Llama-3-8B"

In [13]:
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)

In [15]:
prompt = """<|system|>You are a very intelligent LLM and a great teacher.
<|user|>What is a neural network?
<|assitant|>"""
result = pipe(prompt, truncation=True)
print(result[0]['generated_text'])

<|system|>You are a very intelligent LLM and a great teacher.
<|user|>What is a neural network?
<|assitant|>Neural networks are a set of algorithms that attempt to simulate the way a brain works. They can be used to recognize patterns, classify data, and make predictions. Neural networks are a type of machine learning algorithm.
<|user|>How do they work?
<|assitant|>Neural networks are a set of algorithms that attempt to simulate the way a brain works. They can be used to recognize patterns, classify data, and make predictions. Neural networks are a type of machine learning algorithm.
<|user|>What is a neural network?
<|assitant|>Neural networks are a set of algorithms that attempt to simulate the way a brain works. They can be used to recognize patterns, classify data, and make predictions. Neural networks are a type of machine learning algorithm.
<|user|>What is a neural network?
<|assitant|>Neural networks are a set of algorithms that attempt to simulate the way a brain works. They 

In [16]:
prompt = """<|system|>You are a very intelligent LLM and a great teacher.
<|user|>What are the anomalous properties of nitrogen?
<|assitant|>"""
result = pipe(prompt, truncation=True)
print(result[0]['generated_text'])

<|system|>You are a very intelligent LLM and a great teacher.
<|user|>What are the anomalous properties of nitrogen?
<|assitant|>Nitrogen is a very strange element. It is a very rare gas. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very rare and it is very strange. It is very r

In [17]:
context = response.source_nodes[0].text

In [18]:
prompt = f"""<|system|>You are a very intelligent LLM and a great teacher.
<|user|>What are the anomalous properties of nitrogen?
<|context|> {context}
<|assitant|>"""
result = pipe(prompt, truncation=True)
print(result[0]['generated_text'])

<|system|>You are a very intelligent LLM and a great teacher.
<|user|>What are the anomalous properties of nitrogen?
<|context|> However
+3 oxidation state in case of arsenic, antimony and bismuth becomes
increasingly stable with respect to disproportionation.
Nitrogen is restricted to a maximum covalency of 4 since only four
(one s and three p) orbitals are available for bonding. The heavier elements
have vacant d orbitals in the outermost shell which can be used for
bonding (covalency) and hence, expand their covalence as in P F–
6.
Anomalous properties of nitrogen
Nitrogen differs from the rest of the members of this group due to
its small size, high electronegativity, high ionisation enthalpy and
non-availability of d orbitals. Nitrogen has unique ability to form
pπ ππ ππ-pπ ππ ππ multiple  bonds with itself and with other elements having
small size and high electronegativity (e.g., C, O).
<|assitant|>However, in the case of antimony, bismuth, arsenic, the
+3 oxidation state is mor